In [8]:
import pandas as pd
import pyarrow as pa
import numpy as np
import random
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import torch
import torch.nn.functional as F
from torch import nn
import warnings
warnings.filterwarnings("ignore")

In [24]:
df = pd.read_parquet('/Users/stefanfeiler/Desktop/SmartBetter/SmartBetter/data/stacked_w_opponent_TEST.parquet')

In [25]:
unique_teams = df['team_1'].unique()

In [26]:
df['game_avg_odds'] = np.nan  # Use np.nan to represent missing values

df['game_avg_odds'] = df.groupby(['game_id', 'team_1'])['average_market_odds'].transform('mean')

In [27]:
df['game_id_plus_team'] = df['game_id'] + df['team_1']

In [28]:
my_dict = {game_id: list(group['team_1'].unique()) for game_id, group in df.groupby('game_id')}
game_team_dict = {key: [value for value in value_list if value is not None] for key, value_list in my_dict.items()}

In [29]:
# store each team_game_id prev_odds in a dict: team_1_game_id: [prev_n_odds against this team]
# find all instances of the this team playing that team
# get the average odds for n games below 
# store

In [30]:
# V for team in unique teams 
# V for game played by team 
# V find opponent name with dict lookup (game_id: val)
# V filter the df for just those matchups 
# find n prev games 


In [69]:
my_new_dict = {}
n = 2

for team in unique_teams:
    
    team_df = df[df['team_1'] == team]

    subset_df = team_df.drop_duplicates(subset='game_id')

    unique_opponents = team_df['opponent'].unique()

    for opponent in unique_opponents:
        matchups_df = subset_df[subset_df['opponent'] == opponent]
    
        for i in range(n):
            matchups_df[f'prev_matchup_{i+1}'] = matchups_df['game_avg_odds'].shift(-(i+1))

        for index, row in matchups_df.iterrows():
            my_list = []
            for i in range(n):
                val = row[f'prev_matchup_{i+1}']

                if not np.isnan(val):
                    my_list.append(val)
            
            my_new_dict[row['game_id_plus_team']] = my_list

    my_dict = {key: value if type(value) is list else [value] for key, value in my_new_dict.items()}
    map_dict = {}
    for key, value_list in my_dict.items():
        if len(value_list) > 0:
            average_value  = np.mean(value_list).item()
            map_dict[key] = average_value
        else:
            map_dict[key] = 2

    df['last_n_matchup_avg_odds'] = df['game_id_plus_team'].map(map_dict)

In [70]:
# Testing this
df_rockies = df[df['team_1'] == 'Colorado Rockies']
df_rockies['last_n_matchup_avg_odds']

12128      3.771276
12140      3.771276
12155      3.771276
12171      3.771276
12188      3.771276
             ...   
1734969    2.000000
1734982    2.000000
1734995    2.000000
1735009    2.000000
1735023    2.000000
Name: last_n_matchup_avg_odds, Length: 58116, dtype: float64

In [71]:
df_rockies.to_csv('/Users/stefanfeiler/Desktop/rockies_test_4.csv')